In [3]:
%%writefile fastapiautorestart.py

import subprocess
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

class FastAPIRestartHandler(FileSystemEventHandler):
    def __init__(self, script_path):
        self.script_path = script_path
        self.process = None
        self.start_process()

    def start_process(self):
        # FastAPI 서버를 시작하는 데 사용되는 스크립트 실행
        if self.process:
            self.process.kill()  # 기존 프로세스가 있다면 종료
        self.process = subprocess.Popen(['uvicorn', 'main:app', '--host', '0.0.0.0', '--port', '8000'], stdout=subprocess.PIPE)

    def on_modified(self, event):
        # 특정 파일이 수정되었을 때만 반응
        if event.src_path == self.script_path:
            print(f'Changes detected in {event.src_path}. Restarting FastAPI server...')
            self.start_process()

if __name__ == "__main__":
    path = '/home/j-k10d105/'  # 감시할 파일 또는 디렉토리 경로
    script_file = path+'model.pkl'  # 감시할 특정 파일
    event_handler = FastAPIRestartHandler(script_file)
    observer = Observer()
    observer.schedule(event_handler, path, recursive=False)
    observer.start()

    try:
        while True:
            # 스크립트가 계속 실행되도록 유지
            pass
    except KeyboardInterrupt:
        observer.stop()

    observer.join()


Overwriting fastapiautorestart.py
